In [2]:
import requests
from bs4 import BeautifulSoup

import random
import time

In [3]:
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) rv:45.0 Gecko/20100101 Firefox/45.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (iPad; CPU OS 13_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Linux; Android 11; Pixel 5 Build/RQ2A.210405.005) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.105 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 9; SM-A107F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 8.1.0; Redmi 6A) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.210 Mobile Safari/537.36',
    'Mozilla/5.0 (X11; CrOS x86_64 13982.82.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.130 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 9; LM-Q720) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0',
    'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; U; Android 8.1.0; en-US; C210AE Build/O11019) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/70.0.3538.80 Mobile Safari/537.36',
    'Mozilla/5.0 (X11; Linux i686; rv:64.0) Gecko/20100101 Firefox/64.0',
    'Mozilla/5.0 (Windows NT 10.0; ARM; Surface Pro X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/12.0 Mobile/15A372 Safari/604.1',
    'Mozilla/5.0 (Linux; Android 11; SM-A515F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.86 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:53.0) Gecko/20100101 Firefox/53.0',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 8.0.0; SM-G950F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Mobile Safari/537.36',
    'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0',
    'Mozilla/5.0 (Linux; Android 10; SM-A205U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0',
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/601.7.7 (KHTML, like Gecko) Version/9.1.3 Safari/537.86.7',
    'Mozilla/5.0 (X11; Linux i686; rv:52.0) Gecko/20100101 Firefox/52.0',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64; Trident/7.0; Touch; rv:11.0) like Gecko',
    'Mozilla/5.0 (Linux; Android 11; V2027) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.62 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0.1; vivo 1606) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; ARM64; rv:85.0) Gecko/20100101 Firefox/85.0',
    'Mozilla/5.0 (Linux; Android 11; SM-M315F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.88 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Brave Chrome/89.0.4389.114 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_5 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Mobile/13G36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0'
]

In [4]:
proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}

In [5]:
from stem import Signal
from stem.control import Controller

def get_new_ip():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate(password='billy1990')
        controller.signal(Signal.NEWNYM)

In [6]:
def update_page(page_num):
    url = f"https://medex.com.bd/generics?page={page_num}"
    
    headers = {
        "User-Agent": random.choice(USER_AGENTS)
    }
    get_new_ip()
    response = requests.get(url, headers=headers, proxies=proxies, timeout=5)
    response.raise_for_status()
    html = response.text
    
    return BeautifulSoup(html, "html")

In [7]:
# paragraphs = soup.find_all('a', class_='hoverable-block darker')
# for p in paragraphs:
#     # print(p.text)
#     link = p.get("href")
#     # text = p.text.strip()
#     title_div = p.find("div", class_="col-xs-12 data-row-top dcind-title")
#     title = title_div.text.strip() if title_div else "No title"
    
#     avail_brand_div = p.find("div", class_="col-xs-12 dcind")
#     avail_brand = avail_brand_div.text.replace(" available brands", "").strip() if avail_brand_div else "N/A"

In [8]:
page = 1
data = []

In [ ]:
while True:
    soup = update_page(page)
    
    paragraphs = soup.find_all('a', class_='hoverable-block darker')
    for index, p in enumerate(paragraphs):
        # print(p.text)
        link = p.get("href")
        # text = p.text.strip()
        title_div = p.find("div", class_="col-xs-12 data-row-top dcind-title")
        title = title_div.text.strip() if title_div else "No title"
        
        avail_brand_div = p.find("div", class_="col-xs-12 dcind")
        avail_brand = avail_brand_div.text.replace(" available brands", "").strip() if avail_brand_div else "N/A"

        data.append({
            "Index": index,
            "Title": title,
            "Available Brands": avail_brand,
            "Link": link
        })
    if not paragraphs:
        break

    page += 1

In [9]:
import pandas as pd
medicine_types = pd.DataFrame(data)

In [10]:
medicine_types = pd.read_csv('data/generics/all_title.csv')

In [11]:
medicine_types["Available Brands"] = pd.to_numeric(medicine_types["Available Brands"], errors='coerce')

In [ ]:
import os

# Ensure the directory exists
os.makedirs("data/generics", exist_ok=True)

medicine_types.to_csv("data/generics/all_title.csv", index=False)

All the title of the generics have been scraped. Now, we just have to fetch the medicines only.

In [35]:
def generic_brands_page(url):
    headers = {
        "User-Agent": random.choice(USER_AGENTS)
    }
    time.sleep(5)
    get_new_ip()
    
    response = requests.get(url, headers=headers, proxies=proxies, timeout=5)
    response.raise_for_status()
    html = response.text
    
    return BeautifulSoup(html, "html")

In [13]:
nested_data = []

In [14]:
def fetch_from_main(avail_brands, brand_list):
    for brand in avail_brands:
        a_tag = brand.find("a", class_="hoverable-block brand-item")
        if not a_tag:
            continue
    
        link = a_tag.get("href")
        
        li_tag = a_tag.find("li", class_="data-row")
        if not li_tag:
            continue
            
        name_div = li_tag.find("div", class_="data-row-top")
        name = name_div.text.strip() if name_div else "N/A"
        
        strength_div = li_tag.find("div", class_="data-row-strength")
        strength = strength_div.text.strip() if strength_div else "N/A"
    
        company_div = li_tag.find("div", class_="data-row-company")
        company = company_div.text.strip() if company_div else "N/A"
    
        price_div = li_tag.find("div", class_="packages-wrapper")
        price = price_div.text.strip() if price_div else "N/A"
        
        brand_list.append({
            "Name": name,
            "Strength": strength,
            "Company": company,
            "Price": price,
            "Link": link
        })

In [15]:
def fetch_from_view_more(brand_page_link, brand_list):
    # view_more_link_tag = soup.find("a", string=lambda text: text and "View More Brands" in text)

    # if view_more_link_tag:
    view_more_url = f"{brand_page_link}/brand-names"
    # print("View More URL:", view_more_url)
    # else:
    #     print("View More link not found.")
    
    ##==================================== go to view more if the page is already loaded ================================
    soup_in_tables = generic_brands_page(view_more_url)
    avail_brands_in_tables = soup_in_tables.find('table', class_="table gg-table bindex-table")
    
    if avail_brands_in_tables:
        rows = avail_brands_in_tables.find('tbody').find_all('tr', class_='brand-row')
        
        for row in rows:
            name = row.find('td', {'data-col': 'name'}).get_text(strip=True)
            strength = row.find_all('td')[2].get_text(strip=True)
            company = row.find_all('td')[3].get_text(strip=True)
            price = row.find('td', {'data-col': 'price'}).get_text(strip=True).replace('\n', ' ')
            link = row.get('data-href')
    
            brand_list.append({
                "Name": name,
                "Strength": strength,
                "Company": company,
                "Price": price,
                "Link": link
            })
    
    #==================================== fetched the data fom view more page ============================================

In [24]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_generic(index, brand_page_link, is_link_only = False):
    if pd.isna(brand_page_link):
        return None

    try:
        # Pick the right row once, then pull both fields
        if is_link_only:
            row = medicine_types.iloc[index]
        else:
            # Find the first match by Link (you probably only ever have one)
            row = medicine_types.loc[medicine_types["Link"] == brand_page_link].iloc[0]
        
        parent_title = row["Title"]
        available_brands = row["Available Brands"]

        soup = generic_brands_page(brand_page_link)
        avail_brands = soup.select('div.available-brands')
        brand_list = []

        if available_brands <= 44:
            fetch_from_main(avail_brands, brand_list)
        else:
            fetch_from_view_more(soup, brand_list)

        def get_section_text(soup, section_id):
            section = soup.find("div", id=section_id)
            if section and section.find_next("div", class_="ac-body"):
                return section.find_next("div", class_="ac-body").get_text(strip=True)
            return "N/A"

        return {
            "Title": parent_title,
            "Available Brands": available_brands,
            "Link": brand_page_link,
            "Brands": brand_list,
            "Description": get_section_text(soup, "description"),
            "Indications": get_section_text(soup, "indications"),
            "Composition": get_section_text(soup, "composition"),
            "Pharmacology": get_section_text(soup, "mode_of_action"),
            "Dosage & Administrations": get_section_text(soup, "dosage"),
            "Interaction": get_section_text(soup, "interaction"),
            "Contraindications": get_section_text(soup, "contraindications"),
            "Side Effects": get_section_text(soup, "side_effects"),
            "Pregnancy & Lactation": get_section_text(soup, "pregnancy_cat"),
            "Use in Special Populations": get_section_text(soup, "pediatric_uses"),
            "Precautions & Warnings": get_section_text(soup, "precautions"),
            "Overdose Effects": get_section_text(soup, "overdose_effects"),
            "Therapeutic Class": get_section_text(soup, "drug_classes"),
            "Reconstitution": get_section_text(soup, "reconstitution"),
            "Storage Conditions": get_section_text(soup, "storage_conditions"),
        }
    except Exception as e:
        print(f"[ERROR] Index {index}")
        return {"failed": brand_page_link}

In [17]:
failed_data = []

In [18]:
nested_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [
        executor.submit(process_generic, index, row["Link"])
        for index, row in medicine_types.iterrows()
    ]
    for future in as_completed(futures):
        result = future.result()
        if "failed" in result:
            failed_data.append(result["failed"])
        else:
            nested_data.append(result)

[ERROR] Index 9
[ERROR] Index 36
[ERROR] Index 44
[ERROR] Index 45
[ERROR] Index 46
[ERROR] Index 47
[ERROR] Index 66
[ERROR] Index 82
[ERROR] Index 83
[ERROR] Index 86
[ERROR] Index 91
[ERROR] Index 92
[ERROR] Index 106
[ERROR] Index 103
[ERROR] Index 122
[ERROR] Index 127
[ERROR] Index 147
[ERROR] Index 142
[ERROR] Index 170
[ERROR] Index 173
[ERROR] Index 167
[ERROR] Index 193
[ERROR] Index 200
[ERROR] Index 205
[ERROR] Index 221
[ERROR] Index 229
[ERROR] Index 230
[ERROR] Index 231
[ERROR] Index 232
[ERROR] Index 227
[ERROR] Index 233
[ERROR] Index 234
[ERROR] Index 235
[ERROR] Index 236
[ERROR] Index 237
[ERROR] Index 274
[ERROR] Index 285
[ERROR] Index 292
[ERROR] Index 296
[ERROR] Index 300
[ERROR] Index 293
[ERROR] Index 304
[ERROR] Index 303
[ERROR] Index 302
[ERROR] Index 309
[ERROR] Index 312
[ERROR] Index 332
[ERROR] Index 335
[ERROR] Index 351
[ERROR] Index 352
[ERROR] Index 353
[ERROR] Index 354
[ERROR] Index 355
[ERROR] Index 356
[ERROR] Index 357
[ERROR] Index 358
[ERRO

In [ ]:
# with ThreadPoolExecutor(max_workers=10) as executor:
#     futures = [
#         executor.submit(process_generic, index, row["Link"])
#         for index, row in medicine_types.iterrows()
#     ]
#     for future in as_completed(futures):
#         result = future.result()
#         if "failed" in result:
#             failed_data.append(result["failed"])
#         else:
#             nested_data.append(result)

In [55]:
nested_df = pd.DataFrame(nested_data)
nested_df.to_csv("data/generics/detailed_generic_data.csv", index=False)

In [56]:
import json

with open("data/generics/detailed_generic_data.json", "w", encoding="utf-8") as f:
    json.dump(nested_data, f, ensure_ascii=False, indent=4, default=str)

In [40]:
still_failed = []
retried_successses = []

In [44]:
new_failed = []

In [49]:
# Kick off the retry pass
with ThreadPoolExecutor(max_workers=5) as executor:
    # map each URL to a Future, tagging it with its list‐index for reference
    future_to_url = {
        executor.submit(process_generic, idx, url, True): (idx, url)
        for idx, url in enumerate(new_failed)
    }

    for future in as_completed(future_to_url):
        idx, url = future_to_url[future]
        try:
            result = future.result()
            if "failed" in result:
                # our process_generic returned a failure marker
                still_failed.append(url)
            else:
                retried_successes.append(result)
        except Exception as e:
            # an unhandled exception—capture it for further inspection
            still_failed.append(url)
            print(f"[Retry] Exception for {url}: {e!r}")

In [54]:
nested_data.extend(retried_successes)